# Библиотеки

In [3]:
import numpy as np
import pandas as pd
from PIL import Image

#!pip install --upgrade keras
#!pip install --upgrade tensorflow
#conda install mingw libpython (fot theano)

import keras
from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils

import cv2
from tqdm import tqdm

from sklearn.metrics import fbeta_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Загрузка и обработка данных

In [5]:
core_dir = 'Z:\\Kaggle Amazon Rainforest\\'
#core_dir = 'C:\\Kaggle\\Understanding the Amazon from Space\\'
train_dir = core_dir + 'train-jpg\\'
test_dir = core_dir + 'test-jpg\\'

cathegories = ['agriculture', 'artisinal_mine', 'bare_ground', 
                      'blooming', 'blow_down', 'clear', 'cloudy', 'conventional_mine', 
                      'cultivation', 'habitation', 'haze', 'partly_cloudy', 'primary', 
                      'road', 'selective_logging', 'slash_burn', 'water']

train_data = pd.read_csv(core_dir + 'train.csv')
train_data.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


# Функции

подготовка тренировочной выборки по имени категории
принимает категорию, возвращает массив из 0 и 1 (1 - файл содержит категорию, 0 - иначе)

In [6]:
def TrainingSetByCathegory(cathegory):
    df = pd.DataFrame()
    
    df['image_name'] = train_data.image_name.values
    df['indicator'] = np.zeros(train_data.shape[0])
    
    df.loc[df['image_name'].isin(cathegory_dict[cathegory]), ['indicator']] = 1
    
    return df.indicator.values

формирование полной (по всем категориям) тестовой выборки

In [7]:
def TestSetWhole(data, col_name = 'tags'):
    encoder = LabelEncoder()
    encoder.fit(cathegories)
    
    result = np.zeros((data.shape[0], len(cathegories)))
    
    for i in range(data.shape[0]):
        tag = data[col_name][i]
        tag = tag.split(' ')
        v = encoder.transform(tag)
        
        for j in v:
            result[i, j] = 1
    
    return(result)

функция предназначенная для конвертации .jpg изображения в numpy массив
если transparency = False, то каждая точка представляется 3 числами иначе - 4

In [8]:
def ImageToNumpy(img_name, img_dir = train_dir, img_type = '.jpg', transparency = False):
    img = Image.open(train_dir + img_name + img_type)
    img.load()
    data = np.asarray( img, dtype="int32" )
    img.close()
    
    if transparency:
        return data
    else:
        return data[:, :, 0:3]

# Вычисления

создадим словарь категорий
каждой категории будет соответствовать список файлов, в которых данная категория присутствует

In [9]:
cathegory_dict = {cathegory: [] for cathegory in cathegories}

ind = 0
for tag in train_data.tags.values:
    file_name = train_data.image_name[ind]
    
    cathegory_names_list = tag.split(' ')
    
    for cathegory in cathegory_names_list:
        cathegory_dict[cathegory].append(file_name)
    
    ind += 1   

формирование обучающей выборки

In [10]:
X = []

for img_name in tqdm(train_data.image_name.values):
    
    img = cv2.imread(train_dir + img_name + '.jpg')
    res = cv2.resize(img, (32, 32))
    X.append(res)
    
X = np.array(X, np.float16) / 255.

100%|████████████████████████████████████| 40479/40479 [15:50<00:00, 42.59it/s]


формирование тестовой выборки

In [11]:
Y = TestSetWhole(train_data)

разбиение выборки на обучение и тест

In [12]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Настройка и  обучение сети для одной категории

In [18]:
batch_size = 128 # in each iteration, we consider 32 training examples at once
num_epochs = 100 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [14]:
# загрузка изображений и их приведение к подходящему для обработки виду
num_train = 40479
depth = 32 
height = 32
width = 3 

num_classes = len(cathegories)

#y_train = TrainingSetByCathegory(cathegories[0]) #y_train labels (для категории_0 1, если подходит, 0-иначе)
#Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
#X_train = X

In [ ]:
# переделать
'''
def f2_metric(y_true, y_pred):
    result = []
    for i in range(y_true.shape[0]):
        result.append(fbeta_score(y_true[i, :], y_pred[i, :], 2))
    
    result = np.mean(result)
    
    return(result)
'''

'\ndef f2_metric(y_true, y_pred):\n    result = []\n    for i in range(y_true.shape[0]):\n        result.append(fbeta_score(y_true[i, :], y_pred[i, :], 2))\n    \n    result = np.mean(result)\n    \n    return(result)\n'

In [19]:
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!

# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)

conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)

# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)

conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, border_mode='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)

# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax

flat = Flatten()(drop_2)

hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)

out = Dense(num_classes, activation='sigmoid')(drop_3)

model = Model(input=inp, output=out) # To define a model, just specify its input and output layers

model.compile(loss='binary_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(x_train, y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.2) # ...holding out 10% of the data for validation

#model.evaluate(x_test, y_test, verbose=1) # Evaluate the trained model on the test set!
cnn_prediction = model.predict(x_test)

C:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  """
C:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", padding="same")`
  
C:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  if sys.path[0] == '':
C:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", padding="same")`
  del sys.path[0]
C:\Users\horch\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(i

Train on 22668 samples, validate on 5667 samples
Epoch 1/100
22668/22668 [==============================] - ETA: 343s - loss: 0.7006 - acc: 0.426 - ETA: 269s - loss: 0.6611 - acc: 0.651 - ETA: 242s - loss: 0.6049 - acc: 0.729 - ETA: 230s - loss: 0.5463 - acc: 0.770 - ETA: 219s - loss: 0.5065 - acc: 0.794 - ETA: 212s - loss: 0.4951 - acc: 0.808 - ETA: 209s - loss: 0.4776 - acc: 0.819 - ETA: 206s - loss: 0.4618 - acc: 0.824 - ETA: 201s - loss: 0.4466 - acc: 0.830 - ETA: 199s - loss: 0.4348 - acc: 0.834 - ETA: 196s - loss: 0.4257 - acc: 0.837 - ETA: 194s - loss: 0.4192 - acc: 0.840 - ETA: 192s - loss: 0.4127 - acc: 0.844 - ETA: 190s - loss: 0.4060 - acc: 0.848 - ETA: 187s - loss: 0.3994 - acc: 0.851 - ETA: 186s - loss: 0.3922 - acc: 0.855 - ETA: 183s - loss: 0.3865 - acc: 0.858 - ETA: 182s - loss: 0.3825 - acc: 0.860 - ETA: 181s - loss: 0.3767 - acc: 0.863 - ETA: 179s - loss: 0.3722 - acc: 0.865 - ETA: 177s - loss: 0.3687 - acc: 0.867 - ETA: 176s - loss: 0.3661 - acc: 0.867 - ETA: 175s - 

22668/22668 [==============================] - ETA: 217s - loss: 0.2146 - acc: 0.917 - ETA: 215s - loss: 0.2133 - acc: 0.920 - ETA: 215s - loss: 0.2128 - acc: 0.920 - ETA: 215s - loss: 0.2095 - acc: 0.921 - ETA: 215s - loss: 0.2093 - acc: 0.920 - ETA: 213s - loss: 0.2087 - acc: 0.921 - ETA: 213s - loss: 0.2119 - acc: 0.919 - ETA: 212s - loss: 0.2120 - acc: 0.919 - ETA: 210s - loss: 0.2087 - acc: 0.921 - ETA: 209s - loss: 0.2064 - acc: 0.922 - ETA: 208s - loss: 0.2071 - acc: 0.922 - ETA: 206s - loss: 0.2071 - acc: 0.922 - ETA: 205s - loss: 0.2070 - acc: 0.922 - ETA: 204s - loss: 0.2061 - acc: 0.922 - ETA: 203s - loss: 0.2054 - acc: 0.922 - ETA: 201s - loss: 0.2050 - acc: 0.923 - ETA: 200s - loss: 0.2046 - acc: 0.923 - ETA: 199s - loss: 0.2052 - acc: 0.922 - ETA: 198s - loss: 0.2060 - acc: 0.922 - ETA: 197s - loss: 0.2055 - acc: 0.922 - ETA: 196s - loss: 0.2053 - acc: 0.922 - ETA: 194s - loss: 0.2062 - acc: 0.921 - ETA: 193s - loss: 0.2064 - acc: 0.921 - ETA: 192s - loss: 0.2049 - acc: 0

22668/22668 [==============================] - ETA: 241s - loss: 0.1885 - acc: 0.925 - ETA: 240s - loss: 0.1869 - acc: 0.923 - ETA: 239s - loss: 0.1858 - acc: 0.923 - ETA: 237s - loss: 0.1840 - acc: 0.923 - ETA: 236s - loss: 0.1789 - acc: 0.927 - ETA: 233s - loss: 0.1823 - acc: 0.926 - ETA: 232s - loss: 0.1823 - acc: 0.926 - ETA: 231s - loss: 0.1806 - acc: 0.927 - ETA: 230s - loss: 0.1813 - acc: 0.927 - ETA: 228s - loss: 0.1827 - acc: 0.927 - ETA: 226s - loss: 0.1816 - acc: 0.927 - ETA: 225s - loss: 0.1828 - acc: 0.926 - ETA: 224s - loss: 0.1825 - acc: 0.927 - ETA: 222s - loss: 0.1826 - acc: 0.926 - ETA: 221s - loss: 0.1838 - acc: 0.925 - ETA: 219s - loss: 0.1847 - acc: 0.925 - ETA: 218s - loss: 0.1846 - acc: 0.925 - ETA: 217s - loss: 0.1849 - acc: 0.925 - ETA: 215s - loss: 0.1842 - acc: 0.925 - ETA: 214s - loss: 0.1857 - acc: 0.925 - ETA: 212s - loss: 0.1857 - acc: 0.925 - ETA: 211s - loss: 0.1855 - acc: 0.925 - ETA: 210s - loss: 0.1857 - acc: 0.925 - ETA: 208s - loss: 0.1858 - acc: 0

22668/22668 [==============================] - ETA: 243s - loss: 0.1809 - acc: 0.928 - ETA: 243s - loss: 0.1802 - acc: 0.930 - ETA: 243s - loss: 0.1834 - acc: 0.928 - ETA: 243s - loss: 0.1830 - acc: 0.928 - ETA: 241s - loss: 0.1815 - acc: 0.929 - ETA: 239s - loss: 0.1840 - acc: 0.928 - ETA: 238s - loss: 0.1823 - acc: 0.929 - ETA: 236s - loss: 0.1825 - acc: 0.929 - ETA: 234s - loss: 0.1806 - acc: 0.930 - ETA: 233s - loss: 0.1812 - acc: 0.930 - ETA: 232s - loss: 0.1794 - acc: 0.931 - ETA: 231s - loss: 0.1781 - acc: 0.931 - ETA: 229s - loss: 0.1789 - acc: 0.931 - ETA: 228s - loss: 0.1793 - acc: 0.930 - ETA: 226s - loss: 0.1774 - acc: 0.931 - ETA: 225s - loss: 0.1774 - acc: 0.931 - ETA: 224s - loss: 0.1762 - acc: 0.931 - ETA: 223s - loss: 0.1760 - acc: 0.931 - ETA: 221s - loss: 0.1758 - acc: 0.931 - ETA: 220s - loss: 0.1746 - acc: 0.931 - ETA: 219s - loss: 0.1742 - acc: 0.932 - ETA: 217s - loss: 0.1731 - acc: 0.932 - ETA: 216s - loss: 0.1725 - acc: 0.932 - ETA: 215s - loss: 0.1740 - acc: 0

...

22668/22668 [==============================] - ETA: 270s - loss: 0.0540 - acc: 0.980 - ETA: 269s - loss: 0.0487 - acc: 0.981 - ETA: 266s - loss: 0.0457 - acc: 0.983 - ETA: 265s - loss: 0.0460 - acc: 0.982 - ETA: 263s - loss: 0.0434 - acc: 0.983 - ETA: 262s - loss: 0.0419 - acc: 0.984 - ETA: 261s - loss: 0.0410 - acc: 0.984 - ETA: 260s - loss: 0.0416 - acc: 0.984 - ETA: 258s - loss: 0.0413 - acc: 0.984 - ETA: 257s - loss: 0.0424 - acc: 0.984 - ETA: 256s - loss: 0.0419 - acc: 0.984 - ETA: 255s - loss: 0.0429 - acc: 0.984 - ETA: 254s - loss: 0.0419 - acc: 0.984 - ETA: 252s - loss: 0.0426 - acc: 0.984 - ETA: 250s - loss: 0.0428 - acc: 0.983 - ETA: 249s - loss: 0.0428 - acc: 0.983 - ETA: 248s - loss: 0.0420 - acc: 0.983 - ETA: 246s - loss: 0.0416 - acc: 0.983 - ETA: 245s - loss: 0.0420 - acc: 0.983 - ETA: 243s - loss: 0.0426 - acc: 0.983 - ETA: 242s - loss: 0.0426 - acc: 0.983 - ETA: 240s - loss: 0.0429 - acc: 0.983 - ETA: 238s - loss: 0.0430 - acc: 0.983 - ETA: 237s - loss: 0.0429 - acc: 0

22668/22668 [==============================] - ETA: 262s - loss: 0.0298 - acc: 0.988 - ETA: 264s - loss: 0.0351 - acc: 0.986 - ETA: 264s - loss: 0.0345 - acc: 0.986 - ETA: 263s - loss: 0.0370 - acc: 0.986 - ETA: 261s - loss: 0.0370 - acc: 0.985 - ETA: 259s - loss: 0.0354 - acc: 0.986 - ETA: 258s - loss: 0.0361 - acc: 0.986 - ETA: 257s - loss: 0.0364 - acc: 0.986 - ETA: 256s - loss: 0.0382 - acc: 0.985 - ETA: 254s - loss: 0.0391 - acc: 0.984 - ETA: 253s - loss: 0.0397 - acc: 0.984 - ETA: 251s - loss: 0.0401 - acc: 0.984 - ETA: 250s - loss: 0.0406 - acc: 0.984 - ETA: 248s - loss: 0.0406 - acc: 0.984 - ETA: 246s - loss: 0.0409 - acc: 0.984 - ETA: 245s - loss: 0.0405 - acc: 0.984 - ETA: 244s - loss: 0.0406 - acc: 0.984 - ETA: 242s - loss: 0.0401 - acc: 0.984 - ETA: 241s - loss: 0.0400 - acc: 0.984 - ETA: 239s - loss: 0.0402 - acc: 0.984 - ETA: 238s - loss: 0.0402 - acc: 0.984 - ETA: 236s - loss: 0.0397 - acc: 0.984 - ETA: 235s - loss: 0.0397 - acc: 0.984 - ETA: 233s - loss: 0.0396 - acc: 0

22668/22668 [==============================] - ETA: 265s - loss: 0.0451 - acc: 0.981 - ETA: 264s - loss: 0.0409 - acc: 0.983 - ETA: 267s - loss: 0.0409 - acc: 0.983 - ETA: 265s - loss: 0.0403 - acc: 0.984 - ETA: 264s - loss: 0.0408 - acc: 0.984 - ETA: 262s - loss: 0.0406 - acc: 0.984 - ETA: 261s - loss: 0.0414 - acc: 0.983 - ETA: 260s - loss: 0.0404 - acc: 0.984 - ETA: 258s - loss: 0.0400 - acc: 0.984 - ETA: 256s - loss: 0.0409 - acc: 0.983 - ETA: 254s - loss: 0.0418 - acc: 0.983 - ETA: 253s - loss: 0.0412 - acc: 0.983 - ETA: 252s - loss: 0.0430 - acc: 0.983 - ETA: 250s - loss: 0.0433 - acc: 0.982 - ETA: 249s - loss: 0.0436 - acc: 0.982 - ETA: 247s - loss: 0.0434 - acc: 0.982 - ETA: 246s - loss: 0.0432 - acc: 0.982 - ETA: 245s - loss: 0.0430 - acc: 0.983 - ETA: 243s - loss: 0.0433 - acc: 0.982 - ETA: 242s - loss: 0.0434 - acc: 0.982 - ETA: 240s - loss: 0.0430 - acc: 0.982 - ETA: 239s - loss: 0.0427 - acc: 0.983 - ETA: 237s - loss: 0.0428 - acc: 0.983 - ETA: 236s - loss: 0.0429 - acc: 0

22668/22668 [==============================] - ETA: 266s - loss: 0.0210 - acc: 0.992 - ETA: 266s - loss: 0.0358 - acc: 0.987 - ETA: 265s - loss: 0.0394 - acc: 0.985 - ETA: 264s - loss: 0.0405 - acc: 0.986 - ETA: 262s - loss: 0.0407 - acc: 0.985 - ETA: 260s - loss: 0.0422 - acc: 0.984 - ETA: 259s - loss: 0.0436 - acc: 0.984 - ETA: 258s - loss: 0.0439 - acc: 0.983 - ETA: 256s - loss: 0.0438 - acc: 0.983 - ETA: 255s - loss: 0.0434 - acc: 0.983 - ETA: 253s - loss: 0.0438 - acc: 0.983 - ETA: 252s - loss: 0.0433 - acc: 0.983 - ETA: 250s - loss: 0.0444 - acc: 0.983 - ETA: 249s - loss: 0.0438 - acc: 0.983 - ETA: 247s - loss: 0.0439 - acc: 0.983 - ETA: 246s - loss: 0.0441 - acc: 0.983 - ETA: 244s - loss: 0.0444 - acc: 0.983 - ETA: 243s - loss: 0.0444 - acc: 0.983 - ETA: 241s - loss: 0.0452 - acc: 0.982 - ETA: 240s - loss: 0.0450 - acc: 0.982 - ETA: 238s - loss: 0.0450 - acc: 0.982 - ETA: 237s - loss: 0.0456 - acc: 0.982 - ETA: 236s - loss: 0.0454 - acc: 0.982 - ETA: 234s - loss: 0.0453 - acc: 0

22668/22668 [==============================] - ETA: 300s - loss: 0.0524 - acc: 0.981 - ETA: 284s - loss: 0.0442 - acc: 0.983 - ETA: 279s - loss: 0.0455 - acc: 0.981 - ETA: 274s - loss: 0.0480 - acc: 0.981 - ETA: 270s - loss: 0.0488 - acc: 0.981 - ETA: 267s - loss: 0.0502 - acc: 0.980 - ETA: 264s - loss: 0.0497 - acc: 0.981 - ETA: 262s - loss: 0.0506 - acc: 0.980 - ETA: 260s - loss: 0.0485 - acc: 0.981 - ETA: 259s - loss: 0.0472 - acc: 0.982 - ETA: 258s - loss: 0.0467 - acc: 0.981 - ETA: 257s - loss: 0.0468 - acc: 0.981 - ETA: 256s - loss: 0.0461 - acc: 0.981 - ETA: 254s - loss: 0.0463 - acc: 0.981 - ETA: 254s - loss: 0.0464 - acc: 0.981 - ETA: 252s - loss: 0.0460 - acc: 0.982 - ETA: 250s - loss: 0.0462 - acc: 0.982 - ETA: 248s - loss: 0.0459 - acc: 0.982 - ETA: 247s - loss: 0.0454 - acc: 0.982 - ETA: 245s - loss: 0.0451 - acc: 0.982 - ETA: 243s - loss: 0.0449 - acc: 0.982 - ETA: 241s - loss: 0.0452 - acc: 0.982 - ETA: 240s - loss: 0.0447 - acc: 0.982 - ETA: 238s - loss: 0.0445 - acc: 0

In [ ]:
#f2_metric(y_test, cnn_prediction)

In [20]:
# Генерируем описание модели в формате json
model_json = model.to_json()
# Записываем модель в файл
json_file = open("model2.json", "w")
json_file.write(model_json)
json_file.close()

In [21]:
model.save_weights("model2.h5")